In [1]:
import os
from pathlib import Path
from typing import List, Tuple

import cv2 as cv
import gdown
import numpy as np
from tqdm import tqdm

from deepface.extendedmodels.Gender import loadModel
import keras
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib
from matplotlib import pyplot as plt

In [2]:
def cv_imshow(image: np.ndarray, ax=plt) -> None:
    ax.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    ax.axis('off')

def quadrant_bboxes(img_size: Tuple[int, int]) -> List[List[int]]:
    width, height = [d // 2 for d in img_size]
    bboxes = [[x0, y0, x0+width, y0+height] for x0 in [0, width] for y0 in [0, height]]
    return bboxes

def crop_bbox(image: np.ndarray, bbox: np.array) -> np.ndarray:
    x0, y0, x1, y1 = bbox
    return image[y0:y1, x0:x1]

In [3]:
raw_root = Path('Z:/Dev/etienne/SAAC/Midjourney/gender')

labels = sorted(list(map(lambda p: p.stem, raw_root.glob('*'))))
print(f'Labels: {labels}')

aligned_root = Path('../images/aligned/gender')

for label in labels:
    (aligned_root / Path(label)).mkdir(parents=True, exist_ok=True)

Labels: ['female', 'male']


In [4]:
def preprocess_face(img, target_size=(224, 224), grayscale = False, enforce_detection = True, detector_backend = 'opencv', return_region = False, align = True):

    #img might be path, base64 or numpy array. Convert it to numpy whatever it is.
    img = load_image(img)
    base_img = img.copy()

    img, region = detect_face(img = img, detector_backend = detector_backend, grayscale = grayscale, enforce_detection = enforce_detection, align = align)

    #--------------------------

    if img.shape[0] == 0 or img.shape[1] == 0:
        if enforce_detection == True:
            raise ValueError("Detected face shape is ", img.shape,". Consider to set enforce_detection argument to False.")
        else: #restore base image
            img = base_img.copy()

    #--------------------------

    #post-processing
    if grayscale == True:
        img = cv.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #---------------------------------------------------
    #resize image to expected shape

    # img = cv2.resize(img, target_size) #resize causes transformation on base image, adding black pixels to resize will not deform the base image

    if img.shape[0] > 0 and img.shape[1] > 0:
        factor_0 = target_size[0] / img.shape[0]
        factor_1 = target_size[1] / img.shape[1]
        factor = min(factor_0, factor_1)

        dsize = (int(img.shape[1] * factor), int(img.shape[0] * factor))
        img = cv.resize(img, dsize)

        # Then pad the other side to the target size by adding black pixels
        diff_0 = target_size[0] - img.shape[0]
        diff_1 = target_size[1] - img.shape[1]
        if grayscale == False:
            # Put the base image in the middle of the padded image
            img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2), (0, 0)), 'constant')
        else:
            img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2)), 'constant')

    #------------------------------------------

    #double check: if target image is not still the same size with target.
    if img.shape[0:2] != target_size:
        img = cv.resize(img, target_size)

    #---------------------------------------------------

    #normalizing the image pixels

    #img_pixels = image.img_to_array(img) #what this line doing? must?
    #img_pixels = np.expand_dims(img_pixels, axis = 0)
    #img_pixels /= 255 #normalize input in [0, 1]

    #---------------------------------------------------

    if return_region == True:
        return img, region
    else:
        return img

In [5]:
aligned_root.joinpath(*list(raw_root.glob('*/*.png'))[0].parts[-2:])

WindowsPath('../images/aligned/gender/female/alocasia_a_woman_answering_the_phone_photorealistic_0bec091d-fa85-4c0f-b27c-d0d791ec7add_1.png')

In [60]:
detector_backend = 'mtcnn'

for image_path in tqdm(raw_root.glob('*/*.png')):
    try:
        image = cv.imread(str(image_path))
        
        img_224, region = preprocess_face(
            img=image,
            target_size=(224, 224),
            grayscale=False,
            detector_backend=detector_backend,
            enforce_detection=True,
            return_region=True
        )
        
        img_224 = np.squeeze(img_224)
        
        aligned_path = aligned_root.joinpath(*image_path.parts[-2:])
        cv.imwrite(str(aligned_path), img_224)
    except ValueError:
        pass

0it [00:00, ?it/s]

1/1 [==============================] - 0s 73ms/step


1it [00:00,  1.19it/s]

1/1 [==============================] - 0s 47ms/step


2it [00:01,  1.26it/s]

...

855it [15:36,  1.22s/it]

1/1 [==============================] - 0s 169ms/step


856it [15:38,  1.25s/it]

1/1 [==============================] - 0s 153ms/step


857it [15:39,  1.10s/it]


In [6]:
gender_model = loadModel()
gender_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_input (Input  [(None, 224, 224, 3)]    0         
 Layer)                                                          
                                                                 
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                             

In [17]:
from deepface import DeepFace
DeepFace.build_model?

Signature: DeepFace.build_model(model_name)
Docstring:
This function builds a deepface model
Parameters:
        model_name (string): face recognition or facial attribute model
                VGG-Face, Facenet, OpenFace, DeepFace, DeepID for face recognition
                Age, Gender, Emotion, Race for facial attributes

Returns:
        built deepface model
File:      c:\dev\miniconda-envs\windows\saac_p38\lib\site-packages\deepface\deepface.py
Type:      function


In [7]:
gender_model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [8]:
# Use early stopping to optimally terminate training through callbacks
es = EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=2
)

# Save best model automatically
model_path = Path('../../models/gender_model.h5')

mc = ModelCheckpoint(
    filepath=str(model_path),
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode='auto'
)

cb_list = [ 
    es,
    mc
] 

In [9]:
class_names = ['female', 'male']
image_size = (224, 224)
batch_size = 32
seed = 42

midjourney_root = Path('../images/aligned/gender/')

midjourney_ds = keras.utils.image_dataset_from_directory(
    directory=str(midjourney_root),
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=seed
)

Found 847 files belonging to 2 classes.


In [10]:
def dataset_split(dataset, splits=(0.70, 0.15, 0.15)):
    train_split, val_split, test_split = splits
    assert (train_split + val_split + test_split) == 1
    
    ds_size = len(dataset)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = dataset.take(train_size).shuffle(buffer_size=train_size)   
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [11]:
train_ds, val_ds, test_ds = dataset_split(midjourney_ds)

In [12]:
epochs=10

gender_model.fit(train_ds,
                 epochs=epochs,
                 verbose=2,
                 validation_data=val_ds,
                 callbacks=cb_list
                )

Epoch 1/10

Epoch 00001: val_loss improved from inf to 2.75822, saving model to ..\..\models\gender_model.h5
18/18 - 633s - loss: 4265.4072 - accuracy: 0.5295 - val_loss: 2.7582 - val_accuracy: 0.5312 - 633s/epoch - 35s/step
Epoch 2/10

Epoch 00002: val_loss improved from 2.75822 to 0.67790, saving model to ..\..\models\gender_model.h5
18/18 - 605s - loss: 1.9569 - accuracy: 0.5052 - val_loss: 0.6779 - val_accuracy: 0.4922 - 605s/epoch - 34s/step
Epoch 3/10

Epoch 00003: val_loss improved from 0.67790 to 0.54220, saving model to ..\..\models\gender_model.h5
18/18 - 607s - loss: 0.6542 - accuracy: 0.6302 - val_loss: 0.5422 - val_accuracy: 0.7969 - 607s/epoch - 34s/step
Epoch 4/10

Epoch 00004: val_loss improved from 0.54220 to 0.54042, saving model to ..\..\models\gender_model.h5
18/18 - 609s - loss: 0.5710 - accuracy: 0.7066 - val_loss: 0.5404 - val_accuracy: 0.7266 - 609s/epoch - 34s/step
Epoch 5/10

Epoch 00005: val_loss did not improve from 0.54042
18/18 - 603s - loss: 0.6348 - accu